# 問題80

問題51の再現

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_root_dir="./drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm NewsAggregatorDataset.zip
!wget --no-check-certificate -q "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"
!unzip -n NewsAggregatorDataset.zip
!rm -rf __MACOSX
!pip install stemming
!pip install spacy
!pip install numpy

rm: cannot remove 'NewsAggregatorDataset.zip': No such file or directory
Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   
  Created wheel for stemming: filename=stemming-1.0.1-cp36-none-any.whl size=11139 sha256=d22a6ea39892f18e02f3a8b2bb12ca39b48e4437a1707782d142d9d5a76c2d10
  Stored in directory: /root/.cache/pip/wheels/e8/05/2e/2ddeb64d4464b854b48323f9676528c17560da7d153db7b0e2
Successfully built stemming


In [ ]:
import spacy
import gensim
import numpy as np
import re
from stemming.porter2 import stem
import pickle

In [ ]:
class Article:
  def __init__(self,ID,TITLE,CATEGORY):
    self.id=ID
    self.title=TITLE
    self.category=CATEGORY
    self.feature_unigram=[]
  def get_title(self):
    return self.title
  def set_unigram(self,library,nlp):
        word_dic={}
        for word in nlp.make_doc(self.title):
            tmp=stem(word.lemma_.lower())
            word_dic.setdefault(tmp,0)
            word_dic[tmp]+=1
        for word in library:
            if word in word_dic.keys():
                self.feature_unigram.append(word_dic[word])
            else:
                self.feature_unigram.append(0)

drive_colab=drive_root_dir+"/Colab Notebooks"
drive_ch9=drive_root_dir+"/Colab Notebooks/ch9"
test_articles=[]
train_articles=[]
valid_articles=[]
with open(drive_colab+"/test.txt") as test, open(drive_colab+"/train.txt") as train, open(drive_colab+"/valid.txt") as valid:
  for line in test.readlines():
    elms=line.split('\t')
    article=Article(elms[0],elms[1],elms[4])
    test_articles.append(article)
  for line in train.readlines():
    elms=line.split('\t')
    article=Article(elms[0],elms[1],elms[4])
    train_articles.append(article)
  for line in valid.readlines():
    elms=line.split('\t')
    article=Article(elms[0],elms[1],elms[4])
    valid_articles.append(article)

In [ ]:
nlp=spacy.load('en')
def create_text_word_dic(articles,nlp):
    word_dic={}
    for article in articles:
        for word in nlp.make_doc(article.get_title()):
            tmp=stem(word.lemma_.lower())
            word_dic.setdefault(tmp,0)
            word_dic[tmp]+=1
    word_dic2={}
    for key,value in word_dic.items():
        if 2<value:
            word_dic2.setdefault(key,value)
    return word_dic2
library=[]
for word in create_text_word_dic(train_articles,nlp):
    library.append(word)
for article in train_articles:
    article.set_unigram(library,nlp)
for article in valid_articles:
    article.set_unigram(library,nlp)
for article in test_articles:
    article.set_unigram(library,nlp)

In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/train_articles.pickle",'wb') as f:
  pickle.dump(train_articles,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_articles.pickle",'wb') as f:
  pickle.dump(test_articles,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_articles.pickle",'wb') as f:
  pickle.dump(valid_articles,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/library.pickle",'wb') as f:
  pickle.dump(library,f)

## 本文

In [ ]:
class Article:
  def __init__(self,ID,TITLE,CATEGORY):
    self.id=ID
    self.title=TITLE
    self.category=CATEGORY
    self.feature_unigram=[]
  def get_title(self):
    return self.title
  def set_unigram(self,library,nlp):
        word_dic={}
        for word in nlp.make_doc(self.title):
            tmp=stem(word.lemma_.lower())
            word_dic.setdefault(tmp,0)
            word_dic[tmp]+=1
        for word in library:
            if word in word_dic.keys():
                self.feature_unigram.append(word_dic[word])
            else:
                self.feature_unigram.append(0)

In [ ]:
import pickle
with open(drive_root_dir+"/Colab Notebooks/ch9/train_articles.pickle",'rb') as f:
  train_articles = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_articles.pickle",'rb') as f:
  test_articles = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_articles.pickle",'rb') as f:
  valid_articles = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/library.pickle",'rb') as f:
  library = pickle.load(f)
nlp=spacy.load('en')

In [ ]:
frequency = [0] * len(library)
frequency = np.array(frequency)

for article in train_articles:
  frequency = frequency + np.array(article.feature_unigram)

word_frequency={}
for word,count in zip(library,frequency):
  word_frequency.setdefault(word,count)

word_frequency = sorted(word_frequency.items(),reverse=True)
word_ID = {}
ID=1
for word,frequency in word_frequency:
  if frequency > 1:
    word_ID.setdefault(word,ID)
    ID += 1
  else :
    word_ID.setdefault(word,0)

def sentence_ID(sentence, word_ID, nlp):
  word_num=[]
  for word in nlp.make_doc(sentence):
    tmp=stem(word.lemma_.lower())
    if tmp in word_ID.keys():
      word_num.append(word_ID[tmp])
    else:
      word_num.append(0)
  return word_num

In [ ]:
sentence_ID("I am Shimokawa.",word_ID,nlp)

[2570, 4191, 0, 4790]

In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'wb') as f:
  pickle.dump(word_ID,f)

# 問題81

In [ ]:
nlp=spacy.load('en')
def sentence_ID(sentence, word_ID, nlp):
  word_num=[]
  for word in nlp.make_doc(sentence):
    tmp=stem(word.lemma_.lower())
    if tmp in word_ID.keys():
      word_num.append(word_ID[tmp])
    else:
      word_num.append(0)
  return word_num
class Article:
  def __init__(self,ID,TITLE,CATEGORY):
    self.id=ID
    self.title=TITLE
    self.category=CATEGORY
    self.feature_unigram=[]
  def get_title(self):
    return self.title
  def set_unigram(self,library,nlp):
        word_dic={}
        for word in nlp.make_doc(self.title):
            tmp=stem(word.lemma_.lower())
            word_dic.setdefault(tmp,0)
            word_dic[tmp]+=1
        for word in library:
            if word in word_dic.keys():
                self.feature_unigram.append(word_dic[word])
            else:
                self.feature_unigram.append(0)
import pickle
with open(drive_root_dir+"/Colab Notebooks/ch9/train_articles.pickle",'rb') as f:
  train_articles = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_articles.pickle",'rb') as f:
  test_articles = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_articles.pickle",'rb') as f:
  valid_articles = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/library.pickle",'rb') as f:
  library = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'rb') as f:
  word_ID = pickle.load(f)

In [ ]:
train_word_ids=[]
for article in train_articles:
  ids = sentence_ID(article.title,word_ID,nlp)
  train_word_ids.append(ids)
test_word_ids=[]
for article in test_articles:
  ids = sentence_ID(article.title,word_ID,nlp)
  test_word_ids.append(ids)
valid_word_ids=[]
for article in valid_articles:
  ids = sentence_ID(article.title,word_ID,nlp)
  valid_word_ids.append(ids)

In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/train_word_ids.pickle",'wb') as f:
  pickle.dump(train_word_ids,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_word_ids.pickle",'wb') as f:
  pickle.dump(test_word_ids,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_word_ids.pickle",'wb') as f:
  pickle.dump(valid_word_ids,f)

## ここまで、id列作成

In [ ]:
cate={'b':0,'t':1,'e':2,'m':3}
train_categories=[]
for article in train_articles:
  train_categories.append(cate[article.category])
test_categories=[]
for article in test_articles:
  test_categories.append(cate[article.category])
valid_categories=[]
for article in valid_articles:
  valid_categories.append(cate[article.category])
with open(drive_root_dir+"/Colab Notebooks/ch9/train_categories.pickle",'wb') as f:
  pickle.dump(train_categories,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_categories.pickle",'wb') as f:
  pickle.dump(test_categories,f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_categories.pickle",'wb') as f:
  pickle.dump(valid_categories,f)

## ここまでカテゴリーリスト作成

In [ ]:
!pip install stemming
!pip install spacy
!pip install numpy

  Created wheel for stemming: filename=stemming-1.0.1-cp36-none-any.whl size=11139 sha256=9be790db02c9a49d8ad97d0e765939e57a6d744f79aadbbdf08ea417264bd102
  Stored in directory: /root/.cache/pip/wheels/e8/05/2e/2ddeb64d4464b854b48323f9676528c17560da7d153db7b0e2
Successfully built stemming


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_root_dir="./drive/My Drive"
import spacy
import gensim
import numpy as np
import re
from stemming.porter2 import stem
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
nlp=spacy.load('en')
with open(drive_root_dir+"/Colab Notebooks/ch9/train_word_ids.pickle",'rb') as f:
  train_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_word_ids.pickle",'rb') as f:
  test_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_word_ids.pickle",'rb') as f:
  valid_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'rb') as f:
  word_ID = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/train_categories.pickle",'rb') as f:
  train_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_categories.pickle",'rb') as f:
  test_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_categories.pickle",'rb') as f:
  valid_categories = pickle.load(f)


In [ ]:
 !pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 753.4MB 23kB/s 
     |████████████████████████████████| 4.0MB 38.8MB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
  def __init__(self,x_size, e_size, h_size, o_size):
    super().__init__()
    self.embed = nn.Embedding(x_size, e_size)
    self.rnn = nn.LSTM(e_size, h_size)
    self.out = nn.Linear(h_size, o_size)

  def predict(self, x, state = None):
    x = self.embed(x.to(torch.int64))
    x, (h, c) = self.rnn(x, state)
    h = F.softmax(self.out(h.squeeze(0)[len(h)-1]))
    return h

  def forward(self, x):
    return self.predict(x)

model = Model(len(word_ID)+1,300,50,4)

In [ ]:
def from_ids_to_onehot_tensor(ids, word_ID):
  tensor_words = torch.zeros([len(ids), len(word_ID)+1])
  for i,word_id in enumerate(ids):
    tensor_words[i] [word_id]= 1 
  return tensor_words

In [ ]:
id_tensor_tmp = from_ids_to_onehot_tensor(train_word_ids[0], word_ID)

In [ ]:
print(model.predict(id_tensor_tmp))
print(train_categories[0])

tensor([0.3205, 0.1854, 0.2700, 0.2241], grad_fn=<SoftmaxBackward>)
2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


# 問題82

## 準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_root_dir="./drive/My Drive"
import gensim
import numpy as np
import re
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from sklearn.metrics import accuracy_score

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/train_word_ids.pickle",'rb') as f:
  train_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_word_ids.pickle",'rb') as f:
  test_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_word_ids.pickle",'rb') as f:
  valid_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'rb') as f:
  word_ID = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/train_categories.pickle",'rb') as f:
  train_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_categories.pickle",'rb') as f:
  test_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_categories.pickle",'rb') as f:
  valid_categories = pickle.load(f)

## 本文

In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn

In [ ]:
class Model(nn.Module):
  def __init__(self, i_size, e_size, h_size, o_size):
    super().__init__()
    self.embed = nn.Embedding(i_size, e_size)
    self.rnn = nn.RNN(e_size, h_size, num_layers=1, batch_first=True)
    self.fc1 = nn.Linear(h_size, o_size)

    nn.init.normal_(self.fc1.weight, -0.1, 0.1)
  
  def forward(self, ids):
    device = self.parameters().__next__().device
    hidden = None
    x = self.embed(ids.to(torch.int64).to(device))
    x, h = self.rnn(x, hidden)
    output = F.softmax(self.fc1(h.squeeze()))
    return output

In [ ]:
class Dummy_dataloader():
  def __init__(self, data, category, batch_size):
    self.data = data
    self.category = category
    self.batch_size = batch_size
    self.count=0
  def get(self):
    datas=[]
    categories =[]
    while self.count*self.batch_size <= len(self.data) :
      if (self.count+1)*self.batch_size > len(self.data):
        output_data = self.data[self.count*self.batch_size:]
        output_category = self.category[self.count*self.batch_size:]
        datas.append(output_data)
        categories.append(output_category)
        self.count = 0
        break
      else:
        output_data = self.data[self.count*self.batch_size:(self.count+1)*self.batch_size]
        output_category = self.category[self.count*self.batch_size:(self.count+1)*self.batch_size]
        datas.append(output_data)
        categories.append(output_category)
        self.count +=1
    return zip(datas, categories)

In [ ]:
def item_sorted(datas, categories):
  dic = []
  for data, category in zip(datas, categories):
    dic.append([category, data])
  dic=sorted(dic, key=lambda x:len(x[1]), reverse=True)
  data_sorted=[]
  cate_sorted=[]
  for item in dic:
    data_sorted.append(item[1])
    cate_sorted.append(item[0])
  return data_sorted, cate_sorted

In [ ]:
def match_padding(ids):
  max_length = max([len(id) for id in ids])
  result = torch.zeros(len(ids), max_length)
  for i in range(len(ids)):
    for j in range(len(ids[i])):
      result[i][j] = ids[i][j]
  return result

In [ ]:
def all_model(datas, categories, model):
  preds=[]
  
  sorted_data, sorted_cate=item_sorted(datas, categories)
  dataloader = Dummy_dataloader(sorted_data, sorted_cate, batch_size)
  for data, category in dataloader.get():
    p_data = match_padding(data)
    for i in model(p_data):  #実質o(n) nはサンプル数
      preds.append(i.detach().numpy().tolist())
  return preds, sorted_cate

def calc_accuracy(datas, categories, model):
  preds,cate =all_model(datas, categories, model)
  return accuracy_score(cate, [np.argmax(np.array(pred)) for pred in preds])

def calc_loss(datas, categories, model):
  CEL = nn.CrossEntropyLoss()
  preds,cate =all_model(datas, categories, model)
  loss = CEL(torch.tensor(preds), torch.tensor(cate))
  return loss.item()



In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = Model(len(word_ID)+1, 300, 50, 4)
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = Variable(match_padding(datas))
    cate = Variable(torch.tensor(categories))
    optimizer.zero_grad()
    
    pred = model(ids)
    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model),
                                        calc_accuracy(train_word_ids, train_categories, model)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model),
                                        calc_accuracy(test_word_ids, test_categories, model)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


epoch:0
	train
		loss1.1491923332214355,acccuracy0.5998128217126814
	test
		loss1.1675426959991455,acccuracy0.5850187265917604
epoch:1
	train
		loss1.0900119543075562,acccuracy0.6541881141787552
	test
		loss1.1138664484024048,acccuracy0.6292134831460674
epoch:2
	train
		loss1.0238181352615356,acccuracy0.7222274216190921
	test
		loss1.0448280572891235,acccuracy0.6928838951310862
epoch:3
	train
		loss0.991697371006012,acccuracy0.7520823584464202
	test
		loss1.0041372776031494,acccuracy0.7415730337078652
epoch:4
	train
		loss0.9840864539146423,acccuracy0.7595694899391671
	test
		loss1.000840425491333,acccuracy0.7460674157303371
epoch:5
	train
		loss0.9720903038978577,acccuracy0.77042583060365
	test
		loss0.9926196336746216,acccuracy0.7475655430711611
epoch:6
	train
		loss0.9554251432418823,acccuracy0.7883013570425831
	test
		loss0.973486065864563,acccuracy0.7662921348314606
epoch:7
	train
		loss0.9458128213882446,acccuracy0.797473093121198
	test
		loss0.9697364568710327,acccuracy0.7700374

# 問題83

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

In [ ]:
def all_model(datas, categories, model, gpu = False):
  preds=[]
  
  sorted_data, sorted_cate=item_sorted(datas, categories)
  dataloader = Dummy_dataloader(sorted_data, sorted_cate, batch_size)
  for data, category in dataloader.get():
    p_data = match_padding(data)
    if gpu:
      p_data = try_gpu(p_data)
    for i in model(p_data):  #実質o(n) nはサンプル数
      preds.append(i.cpu().detach().numpy().tolist())
  return preds, sorted_cate

def calc_accuracy(datas, categories, model, gpu):
  preds,cate =all_model(datas, categories, model)
  return accuracy_score(cate, [np.argmax(np.array(pred)) for pred in preds])

def calc_loss(datas, categories, model, gpu = False):
  CEL = nn.CrossEntropyLoss()
  preds,cate =all_model(datas, categories, model, gpu)
  loss = CEL(torch.tensor(preds), torch.tensor(cate))
  return loss.item()

In [ ]:
model = try_gpu(Model(len(word_ID)+1, 300, 50, 4))
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    
    pred = model(ids)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


epoch:0
	train
		loss1.210397481918335,acccuracy0.554796443612541
	test
		loss1.2257423400878906,acccuracy0.5295880149812734
epoch:1
	train
		loss1.1542720794677734,acccuracy0.5973795039775386
	test
		loss1.1747606992721558,acccuracy0.5775280898876405
epoch:2
	train
		loss1.121736764907837,acccuracy0.6256434253626579
	test
		loss1.1454110145568848,acccuracy0.5970037453183521
epoch:3
	train
		loss1.0822229385375977,acccuracy0.6665418811417876
	test
		loss1.10958993434906,acccuracy0.6299625468164795
epoch:4
	train
		loss1.0480899810791016,acccuracy0.6962096396817969
	test
		loss1.0695960521697998,acccuracy0.6681647940074906
epoch:5
	train
		loss1.0203295946121216,acccuracy0.7226953673373888
	test
		loss1.038426399230957,acccuracy0.7056179775280899
epoch:6
	train
		loss1.008458137512207,acccuracy0.7354234908750585
	test
		loss1.0257203578948975,acccuracy0.7191011235955056
epoch:7
	train
		loss0.9830543994903564,acccuracy0.7604117922321011
	test
		loss0.9979327321052551,acccuracy0.74531835

# 問題84

In [ ]:
!rm GoogleNews-vectors-negative300.bin.gz GoogleNews-vectors-negative300.bin
!wget -q https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
!gunzip -n GoogleNews-vectors-negative300.bin.gz
!pip install gensim

rm: cannot remove 'GoogleNews-vectors-negative300.bin.gz': No such file or directory
rm: cannot remove 'GoogleNews-vectors-negative300.bin': No such file or directory


In [ ]:
import gensim
gen = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
import torch
import pickle
weight = []
for word in word_ID.keys():
  if word in gen:
    weight.append([word_ID[word], gen[word]])
del gen
with open(drive_root_dir+"/Colab Notebooks/ch9/weight.pickle",'wb') as f:
  pickle.dump(weight,f)

## 本文

In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/train_word_ids.pickle",'rb') as f:
  train_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_word_ids.pickle",'rb') as f:
  test_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_word_ids.pickle",'rb') as f:
  valid_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'rb') as f:
  word_ID = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/train_categories.pickle",'rb') as f:
  train_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_categories.pickle",'rb') as f:
  test_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_categories.pickle",'rb') as f:
  valid_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/weight.pickle",'rb') as f:
  weight = pickle.load(f)

In [ ]:
class Model(nn.Module):
  def __init__(self, i_size, e_size, h_size, o_size, embed_weight, gpu=False):
    super().__init__()
    self.embed = nn.Embedding(i_size, e_size)
    self.rnn = nn.RNN(e_size, h_size, num_layers=1, batch_first=True)
    self.fc1 = nn.Linear(h_size, o_size)

    nn.init.normal_(self.fc1.weight, -0.1, 0.1)
    for word_num , weight in embed_weight:
      self.embed.weight.data[word_num] = torch.FloatTensor(weight)
    if gpu:
      try_gpu(self.embed)

  def forward(self, ids):
    device = self.parameters().__next__().device
    hidden = None
    x = self.embed(ids.to(torch.int64).to(device))
    x, h = self.rnn(x, hidden)
    output = F.softmax(self.fc1(h.squeeze()))
    return output

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = try_gpu(Model(len(word_ID)+1, 300, 50, 4, weight, True))
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    
    pred = model(ids)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch:0
	train
		loss1.1777937412261963,acccuracy0.557604117922321
	test
		loss1.1908483505249023,acccuracy0.5423220973782772
epoch:1
	train
		loss1.085305094718933,acccuracy0.6643893308376229
	test
		loss1.1163214445114136,acccuracy0.6202247191011236
epoch:2
	train
		loss1.0249745845794678,acccuracy0.7171736078614881
	test
		loss1.0378226041793823,acccuracy0.701123595505618
epoch:3
	train
		loss0.9975230693817139,acccuracy0.7460926532522227
	test
		loss1.0055290460586548,acccuracy0.7340823970037453
epoch:4
	train
		loss0.9844498634338379,acccuracy0.759288722508189
	test
		loss0.989669919013977,acccuracy0.750561797752809
epoch:5
	train
		loss0.9799051880836487,acccuracy0.7635938231165185
	test
		loss0.9807724952697754,acccuracy0.7647940074906368
epoch:6
	train
		loss0.9742769598960876,acccuracy0.7682732802994853
	test
		loss0.9755774140357971,acccuracy0.7685393258426966
epoch:7
	train
		loss0.968932032585144,acccuracy0.7744501637810014
	test
		loss0.9687515497207642,acccuracy0.77378277

# 問題85

In [ ]:
class Model(nn.Module):
  def __init__(self, i_size, e_size, h_size, o_size, embed_weight, gpu=False):
    super().__init__()
    self.embed = nn.Embedding(i_size, e_size)
    self.rnn = nn.RNN(e_size, h_size, num_layers=1, batch_first=True, bidirectional=True)
    self.fc1 = nn.Linear(h_size*2, o_size)

    nn.init.normal_(self.fc1.weight, -0.1, 0.1)
    for word_num , weight in embed_weight:
      self.embed.weight.data[word_num] = torch.FloatTensor(weight)
    if gpu:
      try_gpu(self.embed)

  def forward(self, ids):
    device = self.parameters().__next__().device
    hidden = None
    x = self.embed(ids.to(torch.int64).to(device))
    x, h = self.rnn(x, hidden)
    output = F.softmax(self.fc1(torch.cat([h[0],h[1]],dim=1)))
    return output

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = try_gpu(Model(len(word_ID)+1, 300, 50, 4, weight, True))
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    
    pred = model(ids)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch:0
	train
		loss1.104232668876648,acccuracy0.6607393542349087
	test
		loss1.1174557209014893,acccuracy0.6411985018726591
epoch:1
	train
		loss1.0459585189819336,acccuracy0.7077211043518952
	test
		loss1.0599994659423828,acccuracy0.6936329588014981
epoch:2
	train
		loss1.0113723278045654,acccuracy0.7351427234440805
	test
		loss1.0204133987426758,acccuracy0.7235955056179775
epoch:3
	train
		loss0.9876043796539307,acccuracy0.7588207767898923
	test
		loss0.9892923831939697,acccuracy0.755056179775281
epoch:4
	train
		loss0.9766773581504822,acccuracy0.7691155825924193
	test
		loss0.9769507646560669,acccuracy0.7700374531835206
epoch:5
	train
		loss0.9697980284690857,acccuracy0.7751988769302761
	test
		loss0.9702441096305847,acccuracy0.7767790262172285
epoch:6
	train
		loss0.9636026620864868,acccuracy0.7809078146934956
	test
		loss0.9636926054954529,acccuracy0.7820224719101123
epoch:7
	train
		loss0.9584278464317322,acccuracy0.7865231633130557
	test
		loss0.959244966506958,acccuracy0.7842

## 多層化

In [ ]:
class Model(nn.Module):
  def __init__(self, i_size, e_size, h_size, o_size, embed_weight, gpu=False):
    super().__init__()
    self.embed = nn.Embedding(i_size, e_size)
    self.rnn = nn.RNN(e_size, h_size, num_layers=2, batch_first=True, bidirectional=True)
    self.fc1 = nn.Linear(h_size*2, o_size)

    nn.init.normal_(self.fc1.weight, -0.1, 0.1)
    for word_num , weight in embed_weight:
      self.embed.weight.data[word_num] = torch.FloatTensor(weight)
    if gpu:
      try_gpu(self.embed)

  def forward(self, ids):
    device = self.parameters().__next__().device
    hidden = None
    x = self.embed(ids.to(torch.int64).to(device))
    x, h = self.rnn(x, hidden)
    output = F.softmax(self.fc1(torch.cat([h[-2],h[-1]],dim=1)))
    return output

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = try_gpu(Model(len(word_ID)+1, 300, 50, 4, weight, True))
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    
    pred = model(ids)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch:0
	train
		loss1.0611757040023804,acccuracy0.6760879737950398
	test
		loss1.0700427293777466,acccuracy0.6659176029962547
epoch:1
	train
		loss0.9761713147163391,acccuracy0.7661207299953205
	test
		loss0.9702117443084717,acccuracy0.7730337078651686
epoch:2
	train
		loss0.9620311260223389,acccuracy0.7810949929808142
	test
		loss0.9541111588478088,acccuracy0.7887640449438202
epoch:3
	train
		loss0.9558996558189392,acccuracy0.786335985025737
	test
		loss0.9493505358695984,acccuracy0.7962546816479401
epoch:4
	train
		loss0.9520686268806458,acccuracy0.7909218530650445
	test
		loss0.9488604068756104,acccuracy0.7955056179775281
epoch:5
	train
		loss0.9377436637878418,acccuracy0.8071127749181095
	test
		loss0.9403235912322998,acccuracy0.8029962546816479
epoch:6
	train
		loss0.9255290627479553,acccuracy0.8187178287318672
	test
		loss0.9343698024749756,acccuracy0.8104868913857678
epoch:7
	train
		loss0.9141860008239746,acccuracy0.829293401965372
	test
		loss0.9272758364677429,acccuracy0.817

# 問題86

## 準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_root_dir="./drive/My Drive"
import gensim
import numpy as np
import re
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from sklearn.metrics import accuracy_score
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/train_word_ids.pickle",'rb') as f:
  train_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_word_ids.pickle",'rb') as f:
  test_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_word_ids.pickle",'rb') as f:
  valid_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'rb') as f:
  word_ID = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/train_categories.pickle",'rb') as f:
  train_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_categories.pickle",'rb') as f:
  test_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_categories.pickle",'rb') as f:
  valid_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/weight.pickle",'rb') as f:
  weight = pickle.load(f)

In [ ]:
class Dummy_dataloader():
  def __init__(self, data, category, batch_size):
    self.data = data
    self.category = category
    self.batch_size = batch_size
    self.count=0
  def get(self):
    datas=[]
    categories =[]
    while self.count*self.batch_size <= len(self.data) :
      if (self.count+1)*self.batch_size > len(self.data):
        output_data = self.data[self.count*self.batch_size:]
        output_category = self.category[self.count*self.batch_size:]
        datas.append(output_data)
        categories.append(output_category)
        self.count = 0
        break
      else:
        output_data = self.data[self.count*self.batch_size:(self.count+1)*self.batch_size]
        output_category = self.category[self.count*self.batch_size:(self.count+1)*self.batch_size]
        datas.append(output_data)
        categories.append(output_category)
        self.count +=1
    return zip(datas, categories)

In [ ]:
def item_sorted(datas, categories):
  dic = []
  for data, category in zip(datas, categories):
    dic.append([category, data])
  dic=sorted(dic, key=lambda x:len(x[1]), reverse=True)
  data_sorted=[]
  cate_sorted=[]
  for item in dic:
    data_sorted.append(item[1])
    cate_sorted.append(item[0])
  return data_sorted, cate_sorted

In [ ]:
def match_padding(ids):
  max_length = max([len(id) for id in ids])
  result = torch.zeros(len(ids), max_length)
  for i in range(len(ids)):
    for j in range(len(ids[i])):
      result[i][j] = ids[i][j]
  return result

In [ ]:
def all_model(datas, categories, model, gpu = False):
  preds=[]
  
  sorted_data, sorted_cate=item_sorted(datas, categories)
  dataloader = Dummy_dataloader(sorted_data, sorted_cate, batch_size)
  for data, category in dataloader.get():
    p_data = match_padding(data)
    if gpu:
      p_data = try_gpu(p_data)
    for i in model(p_data):  #実質o(n) nはサンプル数
      preds.append(i.cpu().detach().numpy().tolist())
  return preds, sorted_cate

def calc_accuracy(datas, categories, model, gpu = False):
  preds,cate =all_model(datas, categories, model, gpu)
  return accuracy_score(cate, [np.argmax(np.array(pred)) for pred in preds])

def calc_loss(datas, categories, model, gpu = False):
  CEL = nn.CrossEntropyLoss()
  preds,cate =all_model(datas, categories, model, gpu)
  loss = CEL(torch.tensor(preds), torch.tensor(cate))
  return loss.item()


## 本文

In [ ]:
class Model(nn.Module):
  def __init__(self, i_size, e_size, h_size, o_size, embed_weight, gpu=False):
    super().__init__()
    self.embed = nn.Embedding(i_size, e_size)
    self.conv = nn.Conv1d(e_size, h_size, 3, padding=1)
    self.fc1 = nn.Linear(h_size, o_size)

    nn.init.normal_(self.fc1.weight, -0.1, 0.1)
    for word_num , weight in embed_weight:
      self.embed.weight.data[word_num] = torch.FloatTensor(weight)
    if gpu:
      try_gpu(self.embed)

  def forward(self, ids):
    device = self.parameters().__next__().device
    x = self.embed(ids.to(torch.int64).to(device))
    x = F.relu(self.conv(x.permute(0, 2, 1)))
    x = F.max_pool1d(x, x.size(-1))
    x = self.fc1(x.squeeze())
    output = F.softmax(x, dim=1)
    return output

In [ ]:
def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = try_gpu(Model(len(word_ID)+1, 300, 50, 4, weight, True))
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    
    pred = model(ids)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

epoch:0
	train
		loss1.090137243270874,acccuracy0.6495086569957885
	test
		loss1.1107063293457031,acccuracy0.6194756554307116
epoch:1
	train
		loss1.0251168012619019,acccuracy0.729995320542817
	test
		loss1.0347542762756348,acccuracy0.7123595505617978
epoch:2
	train
		loss0.9861987233161926,acccuracy0.7648104819840899
	test
		loss0.9920715689659119,acccuracy0.755056179775281
epoch:3
	train
		loss0.9650214910507202,acccuracy0.7837154890032756
	test
		loss0.9706227779388428,acccuracy0.7827715355805244
epoch:4
	train
		loss0.9548743367195129,acccuracy0.7923256902199345
	test
		loss0.9619455337524414,acccuracy0.7880149812734082
epoch:5
	train
		loss0.9486772418022156,acccuracy0.7981282171268133
	test
		loss0.9580652117729187,acccuracy0.7872659176029962
epoch:6
	train
		loss0.9444313645362854,acccuracy0.8008423022929341
	test
		loss0.9555607438087463,acccuracy0.7910112359550562
epoch:7
	train
		loss0.9411841630935669,acccuracy0.8029012634534394
	test
		loss0.9527860283851624,acccuracy0.7962

# 問題87

問題86で学習しました

# 問題88

In [ ]:
class Model(nn.Module):
  def __init__(self, i_size, e_size, h_size, o_size, embed_weight, gpu=False):
    super().__init__()
    self.embed = nn.Embedding(i_size, e_size)
    self.conv = nn.Conv1d(e_size, h_size, 3, padding=1)
    self.rnn = nn.RNN(e_size, h_size, num_layers=2, batch_first=True, bidirectional=True)
    self.fc1 = nn.Linear(h_size, o_size)
    self.fc2 = nn.Linear(h_size*2, o_size)

    nn.init.normal_(self.fc1.weight, -0.1, 0.1)
    nn.init.normal_(self.fc2.weight, -0.1, 0.1)
    for word_num , weight in embed_weight:
      self.embed.weight.data[word_num] = torch.FloatTensor(weight)
    if gpu:
      try_gpu(self.embed)

  def forward(self, ids):
    device = self.parameters().__next__().device
    x = self.embed(ids.to(torch.int64).to(device))
    x1 = F.relu(self.conv(x.permute(0, 2, 1)))
    x1 = F.max_pool1d(x1, x1.size(-1))
    x1 = self.fc1(x1.squeeze())

    hidden = None
    x = self.embed(ids.to(torch.int64).to(device))
    x2, h = self.rnn(x, hidden)
    x2 = self.fc2(torch.cat([h[-2],h[-1]],dim=1))
    output = F.softmax(x1 + x2, dim=1)
    return output

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = try_gpu(Model(len(word_ID)+1, 300, 50, 4, weight, True))
CEL = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(20):
  for datas, categories in train_dataloader.get():
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    
    pred = model(ids)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

epoch:0
	train
		loss1.0547409057617188,acccuracy0.6940570893776322
	test
		loss1.0793781280517578,acccuracy0.6539325842696629
epoch:1
	train
		loss0.9815253615379333,acccuracy0.7645297145531118
	test
		loss0.9796949028968811,acccuracy0.7647940074906368
epoch:2
	train
		loss0.9646655321121216,acccuracy0.7786616752456715
	test
		loss0.9622383117675781,acccuracy0.7842696629213484
epoch:3
	train
		loss0.9544821977615356,acccuracy0.7897051941974731
	test
		loss0.9553041458129883,acccuracy0.7910112359550562
epoch:4
	train
		loss0.9489613175392151,acccuracy0.7932615816565278
	test
		loss0.9530804753303528,acccuracy0.7895131086142322
epoch:5
	train
		loss0.9441361427307129,acccuracy0.7987833411324287
	test
		loss0.9533540606498718,acccuracy0.7902621722846442
epoch:6
	train
		loss0.9281490445137024,acccuracy0.817313991576977
	test
		loss0.9447715282440186,acccuracy0.797752808988764
epoch:7
	train
		loss0.9127720594406128,acccuracy0.8334113242863828
	test
		loss0.934903085231781,acccuracy0.8044

# 問題89

参考 https://qiita.com/yamaru/items/63a342c844cff056a549

## 準備

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 8.0MB/s 
     |████████████████████████████████| 1.1MB 24.6MB/s 
     |████████████████████████████████| 3.0MB 54.3MB/s 
     |████████████████████████████████| 890kB 56.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=81399629ff8836cdb648d249b7d3c13ea6f6116f823f0f2bcaec67d4ccf20303
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_root_dir="./drive/My Drive"
import gensim
import numpy as np
import re
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from sklearn.metrics import accuracy_score
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn

import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import time
from matplotlib import pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open(drive_root_dir+"/Colab Notebooks/ch9/train_word_ids.pickle",'rb') as f:
  train_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_word_ids.pickle",'rb') as f:
  test_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_word_ids.pickle",'rb') as f:
  valid_word_ids = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/word_ID.pickle",'rb') as f:
  word_ID = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/train_categories.pickle",'rb') as f:
  train_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/test_categories.pickle",'rb') as f:
  test_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/valid_categories.pickle",'rb') as f:
  valid_categories = pickle.load(f)
with open(drive_root_dir+"/Colab Notebooks/ch9/weight.pickle",'rb') as f:
  weight = pickle.load(f)

In [ ]:
class Dummy_dataloader():
  def __init__(self, data, category, batch_size):
    self.data = data
    self.category = category
    self.batch_size = batch_size
    self.count=0
  def get(self):
    datas=[]
    categories =[]
    while self.count*self.batch_size <= len(self.data) :
      if (self.count+1)*self.batch_size > len(self.data):
        output_data = self.data[self.count*self.batch_size:]
        output_category = self.category[self.count*self.batch_size:]
        datas.append(output_data)
        categories.append(output_category)
        self.count = 0
        break
      else:
        output_data = self.data[self.count*self.batch_size:(self.count+1)*self.batch_size]
        output_category = self.category[self.count*self.batch_size:(self.count+1)*self.batch_size]
        datas.append(output_data)
        categories.append(output_category)
        self.count +=1
    return zip(datas, categories)

In [ ]:
def item_sorted(datas, categories):
  dic = []
  for data, category in zip(datas, categories):
    dic.append([category, data])
  dic=sorted(dic, key=lambda x:len(x[1]), reverse=True)
  data_sorted=[]
  cate_sorted=[]
  for item in dic:
    data_sorted.append(item[1])
    cate_sorted.append(item[0])
  return data_sorted, cate_sorted

In [ ]:
def match_padding(ids):
  max_length = max([len(id) for id in ids])
  result = torch.zeros(len(ids), max_length, dtype=torch.long)
  for i in range(len(ids)):
    for j in range(len(ids[i])):
      result[i][j] = ids[i][j]
  return result

In [ ]:
def all_model(datas, categories, model, gpu = False):
  preds=[]
  
  sorted_data, sorted_cate=item_sorted(datas, categories)
  dataloader = Dummy_dataloader(sorted_data, sorted_cate, batch_size)
  for data, category in dataloader.get():
    mask = torch.tensor(create_mask(data), dtype=torch.long)
    p_data = match_padding(data)
    if gpu:
      mask = try_gpu(mask)
      p_data = try_gpu(p_data)
    for i in model(p_data, mask):  #実質o(n) nはサンプル数
      preds.append(i.cpu().detach().numpy().tolist())
  return preds, sorted_cate

def calc_accuracy(datas, categories, model, gpu = False):
  preds,cate =all_model(datas, categories, model, gpu)
  return accuracy_score(cate, [np.argmax(np.array(pred)) for pred in preds])

def calc_loss(datas, categories, model, gpu = False):
  CEL = nn.CrossEntropyLoss()
  preds,cate =all_model(datas, categories, model, gpu)
  loss = CEL(torch.tensor(preds), torch.tensor(cate))
  return loss.item()


In [ ]:
def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

## 本文

In [ ]:
class Model(torch.nn.Module):
  def __init__(self, otuput_size):
    super().__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.fc = torch.nn.Linear(768, otuput_size)  # BERTの出力に合わせて768次元を指定

  def forward(self, ids, mask):
    _, out = self.bert(ids, attention_mask=mask)
    out = self.fc(out)
    out = F.softmax(out)
    return out

In [ ]:
def create_mask(batchsize_ids):
  maxlength = max([len(ids) for ids in batchsize_ids])
  return [ [1] * len(ids) + [0]*(maxlength-len(ids))  for ids in batchsize_ids]

In [ ]:
batch_size = 32
train_sorted_data, train_sorted_cate=item_sorted(train_word_ids, train_categories)
train_dataloader = Dummy_dataloader(train_sorted_data, train_sorted_cate, batch_size)

In [ ]:
model = try_gpu(Model(4))
CEL = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

for epoch in range(40):
  for datas, categories in train_dataloader.get():
    mask = try_gpu(torch.tensor(create_mask(datas), dtype=torch.long))
    ids = try_gpu(Variable(match_padding(datas)))
    cate = try_gpu(Variable(torch.tensor(categories)))
    optimizer.zero_grad()
    pred = model(ids, mask)

    loss = CEL(pred, cate)
    loss.backward()
    optimizer.step()
  print("epoch:{}".format(epoch))
  print("\ttrain")
  print("\t\tloss{},acccuracy{}".format(calc_loss(train_word_ids, train_categories, model, gpu=True),
                                        calc_accuracy(train_word_ids, train_categories, model, gpu=True)))
  print("\ttest")
  print("\t\tloss{},acccuracy{}".format(calc_loss(test_word_ids, test_categories, model, gpu=True),
                                        calc_accuracy(test_word_ids, test_categories, model, gpu=True)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


epoch:0
	train
		loss1.106292724609375,acccuracy0.6339728591483388
	test
		loss1.1176788806915283,acccuracy0.6157303370786517
epoch:1
	train
		loss0.9820935726165771,acccuracy0.7597566682264857
	test
		loss0.9869040250778198,acccuracy0.755056179775281
epoch:2
	train
		loss0.9486721158027649,acccuracy0.7861488067384184
	test
		loss0.9702041745185852,acccuracy0.7730337078651686
epoch:3
	train
		loss0.9441455602645874,acccuracy0.7957884885353299
	test
		loss0.9761539697647095,acccuracy0.7588014981273409
epoch:4
	train
		loss1.0052461624145508,acccuracy0.7327094057089377
	test
		loss1.041250228881836,acccuracy0.6928838951310862
epoch:5
	train
		loss0.9206914305686951,acccuracy0.8206832007487131
	test
		loss0.9565999507904053,acccuracy0.7812734082397004
epoch:6
	train
		loss0.9362585544586182,acccuracy0.8029948525970987
	test
		loss0.982696533203125,acccuracy0.7558052434456929
epoch:7
	train
		loss0.961430013179779,acccuracy0.7752924660739354
	test
		loss1.0037000179290771,acccuracy0.731835